In [ ]:
# INSTALLATION / MISE À JOUR COMFYUI AUTOMATIQUE

import os
import subprocess

COMFY_PATH = "/workspace/ComfyUI"

print("🚀 Vérification de l'installation de ComfyUI...")

if os.path.exists(COMFY_PATH):
    print("📂 ComfyUI détecté. Mise à jour en cours...")
    # On force la mise à jour vers la dernière version
    try:
        # git fetch récupère les infos, git reset --hard force l'état local à correspondre exactement au serveur
        subprocess.run(f"cd {COMFY_PATH} && git fetch origin && git reset --hard origin/master", shell=True, check=True)
        print("✅ Mise à jour réussie (Dernier commit master).")
    except Exception as e:
        print(f"⚠️ Erreur lors de la mise à jour : {e}")
        print("   -> Tentative de git pull classique...")
        subprocess.run(f"cd {COMFY_PATH} && git pull", shell=True)
else:
    print("📥 Première installation (Clonage)...")
    subprocess.run(f"git clone https://github.com/comfyanonymous/ComfyUI.git {COMFY_PATH}", shell=True)
    print("✅ Installation terminée.")

In [ ]:
# === INSTALLATION TOTALE FLUX (Nodes + Modèles Vision + Dépendances + Fix NSFW) ===
# FIX: Ajout de l'installation système aria2 et fallback sur wget

import os
import subprocess
import sys

COMFY_DIR = "/workspace/ComfyUI"
CUSTOM_NODES_DIR = f"{COMFY_DIR}/custom_nodes"

print("🚀 DÉMARRAGE DE L'INSTALLATION FLUX & TOOLS...")

# --- 0. PRÉ-REQUIS SYSTÈME (FIX POUR ARIA2C) ---
print("🛠️ Vérification des outils système...")
try:
    # On tente d'installer aria2 silencieusement
    subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    subprocess.run(["apt-get", "install", "-y", "aria2"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("   ✅ Aria2 installé.")
except Exception as e:
    print("   ⚠️ Impossible d'installer aria2 via apt (pas grave, on utilisera wget).")

# --- 1. LISTE DES NODES (Pack Image Pure) ---
nodes_to_install = [
    "https://github.com/Comfy-Org/ComfyUI-Manager",             
    "https://github.com/rgthree/rgthree-comfy",                 
    "https://github.com/kijai/ComfyUI-KJNodes",                 
    "https://github.com/cubiq/ComfyUI_essentials",              
    "https://github.com/negaga53/comfyui-imgloader",            
    "https://github.com/pythongosssss/ComfyUI-Custom-Scripts",  
    "https://github.com/sonnybox/ComfyUI-SuperNodes",           
    "https://github.com/ltdrdata/ComfyUI-Impact-Pack",          
    "https://github.com/ltdrdata/ComfyUI-Impact-Subpack",       
    "https://github.com/numz/ComfyUI-SeedVR2_VideoUpscaler",    
    "https://github.com/ClownsharkBatwing/RES4LYF",             
    "https://github.com/giriss/comfy-image-saver",              
    "https://github.com/Gourieff/ComfyUI-ReActor",              
    "https://github.com/ltdrdata/was-node-suite-comfyui",       
    "https://github.com/caleboleary/Comfyui-calbenodes", 
    "https://github.com/lldacing/ComfyUI_PuLID_Flux_ll",
]

# --- 2. INSTALLATION DES NODES ---
# Pré-installation InsightFace (Critique pour ReActor)
try:
    import insightface
except ImportError:
    print("📦 Installation de InsightFace & ONNX...")
    subprocess.run([sys.executable, "-m", "pip", "install", "insightface", "onnxruntime-gpu"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

for repo_url in nodes_to_install:
    node_name = repo_url.split("/")[-1]
    node_path = os.path.join(CUSTOM_NODES_DIR, node_name)
    
    if not os.path.exists(node_path):
        print(f"⬇️ Clonage de : {node_name}...")
        try:
            subprocess.run(["git", "clone", repo_url, node_path], check=True, stdout=subprocess.DEVNULL)
            req_path = os.path.join(node_path, "requirements.txt")
            if os.path.exists(req_path):
                print(f"   📦 Dépendances pour {node_name}...")
                subprocess.run([sys.executable, "-m", "pip", "install", "-r", req_path], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except subprocess.CalledProcessError:
            print(f"   ❌ Erreur clone {node_name}")
    else:
        print(f"⏩ [SKIP] {node_name} ok.")

# --- 3. TÉLÉCHARGEMENT MODÈLES ULTRALYTICS (YOLO - Pour Impact Pack) ---
print("\n👁️ Vérification des modèles de détection (YOLO)...")
base_yolo = f"{COMFY_DIR}/models/ultralytics"
bbox_path = os.path.join(base_yolo, "bbox")
segm_path = os.path.join(base_yolo, "segm")
os.makedirs(bbox_path, exist_ok=True)
os.makedirs(segm_path, exist_ok=True)

yolo_models = [
    # BBOX (Carrés)
    ("face_yolov8m.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8m.pt", bbox_path),
    ("hand_yolov8s.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt", bbox_path),
    # SEGM (Masques précis)
    ("person_yolov8m-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8m-seg.pt", segm_path),
    ("yolov8n-seg.pt", "https://huggingface.co/Bingsu/adetailer/resolve/main/yolov8n-seg.pt", segm_path),
    ("face_yolov8m-seg_60.pt", "https://huggingface.co/bobmin/face_yolov8m-seg_60/resolve/main/face_yolov8m-seg_60.pt", segm_path)
]

for name, url, dest in yolo_models:
    final_path = os.path.join(dest, name)
    if not os.path.exists(final_path):
        print(f"   ⬇️ Téléchargement : {name}")
        # Méthode robuste : Essaie Aria2c, sinon Wget
        try:
            subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-d", dest, "-o", name, url], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except (FileNotFoundError, subprocess.CalledProcessError):
            print(f"      ⚠️ Aria2c échoué/absent, utilisation de wget...")
            subprocess.run(["wget", "-q", "-O", final_path, url])

# --- 4. PATCH REACTOR NSFW (AUTOMATIQUE) ---
print("\n🔓 Application du patch ReActor (NSFW Bypass)...")
reactor_path = f"{CUSTOM_NODES_DIR}/ComfyUI-ReActor/scripts/reactor_sfw.py"
bypass_code = """
from PIL import Image
import logging
import os
from scripts.reactor_logger import logger
MODEL_EXISTS = True
def ensure_nsfw_model(nsfwdet_model_path): return True
def nsfw_image(img_data, model_path: str): return False
"""

if os.path.exists(os.path.dirname(reactor_path)):
    try:
        with open(reactor_path, "w") as f: f.write(bypass_code)
        print("   ✅ Patch appliqué sur 'reactor_sfw.py'.")
    except Exception as e:
        print(f"   ❌ Erreur patch : {e}")
else:
    print("   ⚠️ ReActor non trouvé, patch ignoré.")

print("\n🎉 ENVIRONNEMENT FLUX COMPLET ET PRÊT !")
print(">>> Relancez la cellule de lancement (ComfyUI) pour charger tout ça.")

In [6]:
# ComfyUI_PuLID_Flux_ll FIX

import os

# Le chemin exact que vous m'avez donné
file_path = "/workspace/ComfyUI/custom_nodes/comfyui_pulid_flux_ll/pulidflux.py"

print(f"🔧 Tentative de réparation sur : {file_path}")

if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    new_lines = []
    patched = False

    for line in lines:
        # On cherche la ligne de définition de la fonction problématique
        if "def pulid_outer_sample_wrappers_with_override" in line:
            if "**kwargs" in line:
                print("ℹ️ Le fichier semble déjà patché (kwargs présent).")
                patched = True
                new_lines.append(line)
            else:
                # On ajoute **kwargs juste avant la parenthèse fermante et les deux points
                # On remplace "):" par ", **kwargs):"
                # Attention aux espaces éventuels, on utilise rstrip pour être sûr
                line_content = line.rstrip() 
                if line_content.endswith("):"):
                    new_line = line_content[:-2] + ", **kwargs):\n"
                    new_lines.append(new_line)
                    patched = True
                    print("✅ Ligne modifiée avec succès : Ajout de **kwargs")
                else:
                    # Cas rare où la définition est sur plusieurs lignes, on force l'ajout simple
                    # Ceci est une sécurité basique
                    print("⚠️ Format complexe détecté, tentative de remplacement simple...")
                    new_line = line.replace("):", ", **kwargs):")
                    new_lines.append(new_line)
                    patched = True
        else:
            new_lines.append(line)

    if patched:
        # On sauvegarde le fichier modifié
        with open(file_path, 'w', encoding='utf-8') as f:
            f.writelines(new_lines)
        print("💾 Fichier sauvegardé.")
        print("🚀 RÉPARATION TERMINÉE ! Vous devez maintenant redémarrer ComfyUI.")
    else:
        print("❌ Impossible de trouver la ligne exacte à modifier. Vérifiez le contenu du fichier.")

else:
    print(f"❌ Erreur : Le fichier n'existe pas au chemin indiqué : {file_path}")

🔧 Tentative de réparation sur : /workspace/ComfyUI/custom_nodes/comfyui_pulid_flux_ll/pulidflux.py
✅ Ligne modifiée avec succès : Ajout de **kwargs
💾 Fichier sauvegardé.
🚀 RÉPARATION TERMINÉE ! Vous devez maintenant redémarrer ComfyUI.


In [4]:
# FLUX FILES DOWNLOAD

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. CONFIGURATION SÉCURISÉE ---
CIVITAI_TOKEN = os.environ.get("CIVITAI_TOKEN", "")
HF_TOKEN = os.environ.get("HF_TOKEN", "")

# Installation silencieuse de aria2
os.system("apt-get update -y > /dev/null && apt-get install -y aria2 > /dev/null")

# --- 2. BASE DE DONNÉES (Flux Core & LoRAs - Chemins Absolus) ---
db_models = {
    "🌊 Flux Core (UNET & Checkpoints)": [
        ("Flux.1 Krea Dev (FP8 Scaled)", "https://civitai.com/api/download/models/2079678?type=Model&format=SafeTensor&size=pruned&fp=fp8", "/workspace/ComfyUI/models/unet", "flux1KreaDevFP8_fp8SCALED.safetensors", "civitai"),
        ("Flux.1 Dev Officiel (FP8)", "https://huggingface.co/Comfy-Org/FLUX.1-dev-ComfyUI-Repackaged/resolve/main/flux1-dev-fp8.safetensors?download=true", "/workspace/ComfyUI/models/unet", "flux1-dev-fp8.safetensors", "hf"),
    ],
    "⚙️ Flux Dépendances (VAE & CLIP)": [
        ("VAE Flux (ae.safetensors)", "https://huggingface.co/black-forest-labs/FLUX.1-schnell/resolve/main/ae.safetensors?download=true", "/workspace/ComfyUI/models/vae", "ae.safetensors", "hf"),
        ("T5 XXL (FP8) - Encodeur Texte", "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors?download=true", "/workspace/ComfyUI/models/clip", "t5xxl_fp8_e4m3fn.safetensors", "hf"),
        ("CLIP L - Encodeur Texte", "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors?download=true", "/workspace/ComfyUI/models/clip", "clip_l.safetensors", "hf"),
    ],
    "💄 Flux LoRAs (Body & Styles)": [
        ("FC Flux Perfect Busts", "https://civitai.com/api/download/models/1782533?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "FC Flux Perfect Busts.safetensors", "civitai"),
        ("Busty Curvy", "https://civitai.com/api/download/models/1732014?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "busty-curvy.safetensors", "civitai"),
        ("TKTT Steep Slope Tiddies XXL", "https://civitai.com/api/download/models/746019?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "TKTT_SteepSlopeTiddiesXXL_FLUX.safetensors", "civitai"),
        ("Venus Body (Thicc)", "https://civitai.com/api/download/models/2244224?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras", "venus_body_thicc.safetensors", "civitai"),
        ("Olivia 2025 (Privé)", "https://huggingface.co/rafatribe/Personnal_Loras/resolve/main/olivia2025.safetensors?download=true", "/workspace/ComfyUI/models/loras", "olivia2025.safetensors", "hf"),
    ]
}

# --- 3. INTERFACE GRAPHIQUE ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='98%')
checkboxes = {}
output_log = widgets.Output(layout={'border': '1px solid #444', 'height': '200px', 'overflow_y': 'scroll'})

accordion_children = []
titles = []

for category, items in db_models.items():
    vbox_items = []
    # Bouton "Tout cocher"
    select_all = widgets.Button(description="Tout cocher", icon="check-square-o", button_style='info', layout=widgets.Layout(width='150px'))
    def on_select_all(b, cat=category):
        for k, val in checkboxes.items():
            if k.startswith(cat):
                val['cb'].value = True
    select_all.on_click(on_select_all)
    vbox_items.append(select_all)
    
    for item in items:
        name, url, path, filename, src = item
        key = f"{category}|{filename}"
        label = f"{name}"
        cb = widgets.Checkbox(value=False, description=label, style=style, layout=layout)
        checkboxes[key] = {'cb': cb, 'data': item}
        vbox_items.append(cb)
        
    accordion_children.append(widgets.VBox(vbox_items))
    titles.append(category)

accordion = widgets.Accordion(children=accordion_children)
for i, title in enumerate(titles):
    accordion.set_title(i, title)
    accordion.selected_index = 0

btn_download = widgets.Button(
    description='LANCER LE TÉLÉCHARGEMENT (Flux)',
    button_style='success', 
    layout=widgets.Layout(width='100%', height='60px', margin='20px 0px 0px 0px'),
    icon='download'
)

# --- 4. LOGIQUE DE TÉLÉCHARGEMENT ---
def run_download(b):
    output_log.clear_output()
    with output_log:
        print("🚀 Traitement des fichiers Flux...\n")
        count = 0
        for key, val in checkboxes.items():
            if val['cb'].value:
                count += 1
                item = val['data']
                name, url, path, filename, src = item
                
                # 1. Dossier cible
                os.makedirs(path, exist_ok=True)
                
                # 2. Vérification présence (Anti-Overwrite)
                full_path = os.path.join(path, filename)
                if os.path.exists(full_path) and os.path.getsize(full_path) > 1024:
                    print(f"⏩ [SKIP] {name} est déjà là. (0 data utilisée)")
                    continue
                
                if "METTRE_URL" in url:
                    print(f"⚠️ [SKIP] Pas d'URL fournie pour {name}.")
                    continue

                # 3. Auth
                auth = ""
                final_url = url
                if src == "hf":
                    auth = f'--header="Authorization: Bearer {HF_TOKEN}"'
                elif src == "civitai":
                    sep = "&" if "?" in url else "?"
                    final_url = f"{url}{sep}token={CIVITAI_TOKEN}"
                
                print(f"⬇️ Téléchargement de : {name}")
                cmd = f'aria2c -x 16 -s 16 -k 1M -c {auth} -d "{path}" -o "{filename}" "{final_url}"'
                
                ret = os.system(cmd)
                if ret == 0:
                    print(f"   ✅ Terminé.")
                else:
                    print(f"   ⚠️ Erreur (Code {ret}).")
                    
        if count == 0:
            print("⚠️ Aucune case cochée.")
        else:
            print("\n🎉 Opérations terminées.")

btn_download.on_click(run_download)

display(widgets.VBox([
    widgets.Label("🌊 INSTALLATION FLUX.1 (Core & LoRAs)"),
    accordion,
    btn_download,
    output_log
]))

In [ ]:
# DÉMARRAGE COMFYUI (TOUT-EN-UN : FIX PYTORCH + DÉPENDANCES)

import os
import subprocess
import sys

# --- CONFIGURATION ---
COMFY_DIR = "/workspace/ComfyUI"
MANAGER_DIR = f"{COMFY_DIR}/custom_nodes/ComfyUI-Manager"

print("🚀 DÉMARRAGE COMFYUI (Mode Réparation & Mise à jour)...")

# 1. INSTALLATION OUTILS SYSTÈME
print("\n🔧 [1/6] Vérification outils système (ffmpeg, aria2)...")
subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["apt-get", "install", "-y", "ffmpeg", "aria2"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 2. RÉPARATION PYTORCH (Le Fix Critique)
# On force la version 2.5.1 pour éviter l'erreur 'is_compiling'
print("\n🔥 [2/6] Mise à jour du Noyau PyTorch (Cela prend ~1 min)...")
# On vire xformers qui cause souvent des conflits avec les nouvelles versions
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "xformers"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
# Installation de la version stable CUDA 12.4
subprocess.check_call([
    sys.executable, "-m", "pip", "install",
    "torch==2.5.1+cu124", "torchvision==0.20.1+cu124", "torchaudio==2.5.1+cu124",
    "--index-url", "https://download.pytorch.org/whl/cu124"
])

# 3. INSTALLATION DÉPENDANCES PYTHON
print("\n📦 [3/6] Installation des librairies manquantes...")
required_packages = [
    "numpy<2",           # Force NumPy 1.x (Crucial)
    "pydantic-settings", # Fix config
    "sentencepiece",     # Fix T5/Wan/Flux
    "aiohttp",           # Fix Manager
    "einops",            # Fix Core
    "av",                # Fix Vidéo
    "torchsde",          # Fix Samplers
    "scipy",             # Fix Math
    "insightface",       # Fix FaceSwap
    "onnxruntime-gpu",   # Fix FaceSwap
    "kornia",            # Fix Image Proc
    "spandrel",          # Fix Upscale
    "color-matcher"      # Fix Wan Color
]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + required_packages)

# Requirements du Manager
if os.path.exists(f"{MANAGER_DIR}/requirements.txt"):
    print("   -> Dépendances ComfyUI-Manager...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", f"{MANAGER_DIR}/requirements.txt"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 4. TÉLÉCHARGEMENT MODÈLES INSIGHTFACE (AntelopeV2)
print("\n📥 [4/6] Vérification des modèles InsightFace...")
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
os.makedirs(antelope_dir, exist_ok=True)
files_antelope = ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]

for f in files_antelope:
    if not os.path.exists(f"{antelope_dir}/{f}"):
        print(f"   ⬇️ Téléchargement : {f}")
        subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-c", f"https://huggingface.co/okaris/antelopev2/resolve/main/{f}?download=true", "-d", antelope_dir, "-o", f], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 5. CONFIGURATION FINALE
print("\n⚙️ [5/6] Configuration finale...")
# On s'assure que le dossier custom_nodes est propre si besoin (optionnel)

# 6. LANCEMENT
print("\n🚀 [6/6] Lancement du serveur...")

run_script = """#!/bin/bash
cd /workspace/ComfyUI
# Arguments optimisés pour RunPod
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --cache-ram 8 --bf16-unet
"""

with open("run_comfy.sh", "w") as f:
    f.write(run_script)
os.chmod("run_comfy.sh", 0o755)

print("✅ ComfyUI démarre. (Si le terminal bloque ici, c'est bon signe : le serveur tourne !)")
!./run_comfy.sh

🚀 DÉMARRAGE COMFYUI (Mode Réparation & Mise à jour)...

🔧 [1/6] Vérification outils système (ffmpeg, aria2)...

🔥 [2/6] Mise à jour du Noyau PyTorch (Cela prend ~1 min)...
Looking in indexes: https://download.pytorch.org/whl/cu124



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip



📦 [3/6] Installation des librairies manquantes...



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


   -> Dépendances ComfyUI-Manager...

📥 [4/6] Vérification des modèles InsightFace...

⚙️ [5/6] Configuration finale...

🚀 [6/6] Lancement du serveur...
✅ ComfyUI démarre. (Si le terminal bloque ici, c'est bon signe : le serveur tourne !)
[START] Security scan
[ComfyUI-Manager] Using `uv` as Python module for pip operations.
Using Python 3.10.12 environment at: /usr
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-12-16 12:26:15.104
** Platform: Linux
** Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
** Python executable: /usr/bin/python
** ComfyUI Path: /workspace/ComfyUI
** ComfyUI Base Folder Path: /workspace/ComfyUI
** User directory: /workspace/ComfyUI/user
** ComfyUI-Manager config path: /workspace/ComfyUI/user/__manager/config.ini
** Log path: /workspace/ComfyUI/user/comfyui.log
Using Python 3.10.12 environment at: /usr
Using Python 3.10.12 environment at: /usr

Prestartup times for custom nodes:
   0.0 seco